# 实验指导

实验目的：

1. 掌握神经网络的基本原理；
2. 掌握word2vec的基本原理；
3. 掌握Keras的基本用法；
4. 掌握tensorflow的基本用法；
5. 掌握RNN的基本原理；
6. 观察熟悉RNN的两种常见变体(LSTM和GRU)的原理和用法；
7. 熟悉深度学习自然语言处理的基本流程。 

### I 实验描述：

#### 数据： 使用爬虫获得的豆瓣评论数据
#### 目标： 建立机器学习模型，能够对输入的句子自动判断其对应的分值或感情倾向

## II 数据预处理

目的： 将文本信息变成神经网络可以处理的数据格式； 

## 第一部分： 基础理论

## III 神经网络的基本原理

#### Q1: 神经网络的Loss函数的作用为何？

回答： 

衡量神经网络预测值和真实值之间的误差，作为优化的目标函数，计算出来的误差值是反向传播的依据。

#### Q2: 神经网络的激活函数(activation function)起什么作用？ 如果没有激活函数会怎么样？ 

回答：

1. 激活函数对输入数据进行了非线性变换。

2. 若没有激活函数会导致模型过于简单，无法拟合非线性数据的复杂情况。

#### Q3: 神经网络的softmax如何理解， 其作用是什么？ 在`答案`中写出softmax的python表达；


答案：

softmax函数把神经网络输出的logits转化为总和为1的概率分布。

```python
import numpy as np
logits = np.array([y1,y2,y3])
softmax = np.exp(logits)/np.sum(np.exp(logits))
```

#### Q4: 简述 normalized_1 和softmax函数的相同点和不同点， 说明softmax相比normalized_1该函数的优势所在
```python
output = np.array([y1, y2, y3])

normalized_1 = output / np.sum(output)
```

回答：

相同点：normalized_1和softmax函数都是数据归一化的方式。

不同点：softmax在标准化之前对数据进行了指数转换。

softmax的优势：可以处理output中同时存在正负数的情况，数据间评分的概率分布不会在计算时抵消。

#### Q5: 写出crossentropy的函数表达式，说明该函数的作用和意义

回答：

cross_entropy $:= -\sum_{c=1}^{C}y_{c}\log{\hat{y}_{c}}$

在分类问题中对比预测值和真实值衡量它们之间的误差。


### IV 掌握word2vec的基本用法

#### Q6: 说明word2vec要解决的问题背景， 以及word2vec的基本思路， 说明word2vec比起之前方法的优势；

回答：

问题背景：
1. 对于简单的方法，词向量化质量低，没有很好地保持不同词语之间的语义关系。
2. 过于复杂的方法，当需要向量化的词汇太多时计算量过大，效率低。

基本思路：
1. 对比不同模型**NNLM**,**RNNLM**的计算复杂度，非线性隐藏层构成了主要的计算复杂度，需要提出一个更有效率的模型。
2. Log-linear models:**CBOW**和**Skip-gram**,两个模型的网络结构都相同，都是一层隐藏层作线性投影后再用log-linear输出分类结果,不同的是**CBOW**是用前后内容预测当前词，而**Skip-gram**是用当前词预测围绕再其附近的词。

优势：
1. 减少了计算复杂度的同时，提高了模型准确率。
2. 更进一步地，训练出来的词向量能以简单代数运算的方式表达词义，例如Paris - France + Italy = Rome


#### Q7: 说明word2vec的预测目标， predication target, 在答案中写出skip-gram和cbow的预测概率；

回答： 
1.  skip-gram, $\hat{y}= \{P(w_{t-i}|w_t),P(w_{t-i+1}|w_t),\cdots P(w_{t-1}|w_t),P(w_{t+1}|w_t) \cdots ,P(w_{t+i}|w_t)\}$
2.  cbow, $\hat{y} = P(w_t|w_{t-i},w_{t-i+1},\cdots w_{t-1},w_{t-1}\cdots,w_{t+i})$

$i$ is window size

hints: 你可能需要查询latex的基本写法

#### Q8: 请说明word2vec的两种常见优化方法，分别阐述其原理；

回答：


1. Hierachical softmax. 将原本要预测的词汇集合$V$转化为binary tree的搜索路径，每一个词都对应着一条唯一的路径，预测词出现的概率就相当于预测这条路径出现的概率。这样作当更新参数时，复杂度从原来的$O(V)$减少到了$O(\log{V})$，大大较少了计算复杂度。

2.Negative sampling. 由于需要更新的参数太多，在进行优化时仅仅更新正类和抽样地更新某些负类有关的参数，其中更新的负类可以是随机抽样的，也可以是根据词频调整的函数分布抽样。

#### Q9: 请说明word2vec中哈夫曼树的作用；

回答：

word2vec使用hierachical softmax进行优化时，需要把预测的词汇转化为binary tree，这个时候通常会采用huffman binary tree。在构造huffman binary tree时，出现频率越高的词汇约接近根节点，而出现频率越低的词汇越接近叶节点，并且保证每个父节点都有两个子节点。在计算结果和优化word2vec时需要遍历从根节点到每个子节点的唯一路径，而只有很少的情况下会达到深层的叶子节点，所以huffman binary tree提高了模型的运算效率。

#### Q10: 在gensim中如何实现词向量？ 请将gensim中实现词向量的代码置于`答案`中

回答：
```python
from gensim.models import Word2Vec

sentence =['some sentences']

model = Word2Vec(sentences, min_count=1)

```

#### Q11: 请说出除了 skip-gram和cbow的其他4中词向量方法的名字， 并且选取其中两个叙述其基本原理。

回答：
ont-hot,count vector,co-occurrence vector,tf-idf

co-occurence vector:
在由两个词在窗口范围内共同出现次数构成的矩阵，通过svd变换后求得的词向量。

tf-idf: term frequency-inverse document frequency

$TF_i = \frac{c_{i}}{C_d} $ 词i出现在文档中的频率

$IDF_i = \log({\frac{N}{c_{i \in d}}} )$ 总文档/词i出现的文档数

$TF-IDF_i = TF_i * IDF_i$ 

### V 掌握keras的基本用法

#### Q12. 参考keras参考手册，构建一个机器学习模型，该模型能够完成使用DNN(deep neural networks) 实现MNIST数据集的分类；

回答：代码请置于下方：

hints:  keras 序列模型构建 https://keras.io/getting-started/sequential-model-guide/ 

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.optimizers import RMSprop
import numpy as np

In [2]:
batch_size = 128
num_classes = 10
epochs = 20

In [9]:
f=np.load('/Users/mozhiwen/.keras/datasets/mnist/mnist.npz')

In [36]:
x_train,y_train,x_test,y_test = f['x_train'],f['y_train'],f['x_test'],f['y_test']

In [17]:
img_size = 28

In [40]:
def reformat(dataset,labels):
    dataset = dataset.reshape(-1,img_size*img_size).astype('float32')
    dataset = (dataset-np.min(dataset))/(np.max(dataset)-np.min(dataset))
    labels = keras.utils.to_categorical(labels[:,None], num_classes)
    return dataset,labels

In [41]:
x_train,y_train = reformat(x_train,y_train)
x_test,y_test = reformat(x_test,y_test)
print(x_train.shape[0],'train samples')
print(x_test.shape[0],'test samples')

60000 train samples
10000 test samples


In [27]:
model = Sequential()
model.add(Dense(512,activation='relu',input_shape=(img_size*img_size,)))
model.add(Dropout(0.2))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes,activation='softmax'))

In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(),
              metrics = ['accuracy'])

In [45]:
history = model.fit(x_train,y_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
                    validation_data=(x_test,y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 10s 171us/step - loss: 0.2471 - acc: 0.9237 - val_loss: 0.1005 - val_acc: 0.9687
Epoch 2/20
60000/60000 [==============================] - 9s 148us/step - loss: 0.1046 - acc: 0.9691 - val_loss: 0.0744 - val_acc: 0.9763
Epoch 3/20
60000/60000 [==============================] - 9s 152us/step - loss: 0.0769 - acc: 0.9770 - val_loss: 0.0753 - val_acc: 0.9768
Epoch 4/20
60000/60000 [==============================] - 9s 153us/step - loss: 0.0607 - acc: 0.9817 - val_loss: 0.0773 - val_acc: 0.9802
Epoch 5/20
60000/60000 [==============================] - 9s 149us/step - loss: 0.0512 - acc: 0.9846 - val_loss: 0.0796 - val_acc: 0.9789
Epoch 6/20
60000/60000 [==============================] - 9s 154us/step - loss: 0.0433 - acc: 0.9871 - val_loss: 0.0694 - val_acc: 0.9827
Epoch 7/20
60000/60000 [==============================] - 10s 162us/step - loss: 0.0398 - acc: 0.9889 - val_loss: 0.0777 

In [46]:
score = model.evaluate(x_test,y_test,verbose=0)

In [47]:
print('Test loss:',score[0])
print('Test accuracy:',score[1])

Test loss: 0.0973729898917102
Test accuracy: 0.9842


### VI 掌握tensorflow的基本用法

#### Q13: 参考tensorflow的参考手册，构建一个机器学习模型，该模型能够完成使用DNN(deep neural networks)实现MNIST数据集的分类；

回答：代码请置于下方：

hints:tensorflow实现MNIST https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/udacity/2_fullyconnected.ipynb

In [71]:
import tensorflow as tf

In [74]:
graph = tf.Graph()
with graph.as_default():
    #input
    train_data = tf.placeholder(tf.float32,shape=(None,img_size*img_size))
    train_labels = tf.placeholder(tf.float32,shape=(None,num_classes))
    keep_prob = tf.placeholder(tf.float32)
    test_data = tf.constant(x_test)
    test_labels = tf.constant(y_test)
    
    #variables
    w1 = tf.Variable(tf.truncated_normal([img_size * img_size, 512]))
    b1 = tf.Variable(tf.zeros([512]))
    w2 = tf.Variable(tf.truncated_normal([512, 512]))
    b2 = tf.Variable(tf.zeros([512]))
    w3 = tf.Variable(tf.truncated_normal([512, num_classes]))
    b3 = tf.Variable(tf.zeros([num_classes]))
    
    #model
    layer1 = tf.nn.relu(train_data@w1+b1)
    drop_layer1 = tf.nn.dropout(layer1,keep_prob=keep_prob)
    layer2 = tf.nn.relu(drop_layer1@w2+b2)
    drop_layer2 = tf.nn.dropout(layer2,keep_prob=keep_prob)
    logits = drop_layer2@w3+b3
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=train_labels,logits=logits))
    
    optimizer = tf.train.RMSPropOptimizer(0.001,decay=0.0).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    
    test_out1 = tf.nn.relu(test_data@w1+b1)
    test_out2 = tf.nn.relu(test_out1@w2+b2)
    test_logits = test_out2@w3+b3
    test_loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=test_labels,logits=test_logits))
    test_prediction = tf.nn.softmax(test_logits)

In [75]:
def get_batches(dataset,labels,batch_size):
    batch_num = dataset.shape[0] // batch_size
    sample_num = batch_size * batch_num
    for i in range(batch_num):
        x = dataset[i*batch_size:(i+1)*batch_size,:]
        y = labels[i*batch_size:(i+1)*batch_size,:]
        yield x,y

In [76]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [77]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    step = 0
    for i in range(epochs):
        for x,y in get_batches(x_train,y_train,batch_size):
            feed_dict = {train_data:x,train_labels:y,keep_prob:0.8}
            _,l,predictions = session.run([optimizer,loss,train_prediction],
                                                feed_dict=feed_dict)
        print('epoch %d loss: %f accuracy %.1f%%' % (i,l,accuracy(predictions, y)))
        
    print('Test loss: %f accuracy %.1f%%' % (test_loss.eval(),accuracy(test_prediction.eval(), y_test)))
            

epoch 0 loss: 58.746006 accuracy 90.6%
epoch 1 loss: 21.229315 accuracy 93.0%
epoch 2 loss: 25.321468 accuracy 92.2%
epoch 3 loss: 22.862274 accuracy 93.8%
epoch 4 loss: 11.274590 accuracy 94.5%
epoch 5 loss: 7.947142 accuracy 96.9%
epoch 6 loss: 2.745044 accuracy 97.7%
epoch 7 loss: 10.025577 accuracy 96.9%
epoch 8 loss: 1.138198 accuracy 97.7%
epoch 9 loss: 7.077971 accuracy 97.7%
epoch 10 loss: 4.300566 accuracy 96.9%
epoch 11 loss: 0.718770 accuracy 98.4%
epoch 12 loss: 3.604064 accuracy 97.7%
epoch 13 loss: 6.006754 accuracy 96.9%
epoch 14 loss: 3.026232 accuracy 96.9%
epoch 15 loss: 2.748903 accuracy 98.4%
epoch 16 loss: 2.132647 accuracy 98.4%
epoch 17 loss: 2.855407 accuracy 96.1%
epoch 18 loss: 4.587665 accuracy 96.9%
epoch 19 loss: 1.937775 accuracy 96.9%
Test loss: 5.906247 accuracy 96.0%


#### Q14: 参考keras和tensorflow对同一问题的实现，说明keras和tensorflow的异同；

回答：Q12,Q12分别用keras和tensorflow实现了dnn识别mnist数据集。相同点是都需要自己定义模型的输入输出，损失函数和优化器。不同点是相比起tensorflow，keras提供了许多常用的api，实现起来代码量少，代码实现的模型简洁易懂，比较简便省事，不需要对模型和编程有深入的了解也能使用。而tensrflow相比起来更加强大，提供了很多在keras下无法掌控的个性化的参数设置，实现同一个任务代码量比较多，使用要求对模型和框架理解比较深入。

#### Q15: Q12， Q13 的tensorflow 或 keras 模型的训练时准确率和测试集准确率分别是多少？
回答：
tensorflow：训练准确率：96.9% 测试准确率：96.0%
keras：训练准确率：97.4% 测试准确率：98.4%

#### Q16: 训练时准确率大于测试集准确率的现象叫什么名字，在神经网络中如何解决该问题？
回答：过拟合。使用early stopping,l2-regularization,drop out。

#### Q17: 请使用自己的语言简述通过正则化 (regularization)减小过拟合的原理；
回答：

1. early stopping是在训练时观察training set 和 validation set的表现当validation set的表现无明显提升时停止训练。

2. l2-regularizarion,改写loss function为 $loss = f_{loss}+\lambda\lVert \omega \rVert_{2}^{2} $ 其中$f_{loss}$是未加上正则项之前的loss function，$\lambda\lVert \omega \rVert_{2}^{2}$是正则项，$\omega$是模型参数。加上了正则项的训练函数，在训练会权衡误差的下降和参数增加对loss fuction的影响，在下降误差的同时尽量使模型参数变得稀疏，避免过拟合。

3. 训练阶段，在深度学习模型层与层的输出和输入之间按比例随机切断一些链接。

#### Q18: 在tensorflow官方实例中给出的fully connected 神经网络的分类模型中，数据进行了哪些预处理，这些预处理的原因是什么？ 
回答：对数据进行了标准化，标准化后的数据不受features单位影响，训练时收敛得比较快。

### VII 掌握RNN的基本原理

#### Q19: 简述RNN解决的问题所具有的特点；
回答：RNN解决问题的特点是，处理的输入或输出数据都是时间或者序列相关，不同时间位置的数据可能相互关联相互影响。

#### Q20: 写出RNN实现时间或者序列相关的数学实现(见课程slides)；
回答：

Elman network:

$h_t = \sigma_{h}({W_hx_t+U_hh_{t-1}+b_h})$

$y_t = \sigma_y(W_yh_t+b_y)$

Jordan network:

$h_t = \sigma_h(W_hx_t+U_hy_{t-1}+b_h)$

$y_t = \sigma_y(W_yh_t+b_y)$

#### Q20: 简述RNN的两种重要变体的提出原因和基本原理？
回答：

提出原因：

在输入长序列训练RNN时，需要展开计算多次使得RNN变成一个很深层的网络，所以非常容易出现梯度消失或梯度爆炸的问题，而且模型的计算和训练会非常缓慢。另一方面，RNN处理长序列时会丧失长期记忆的能力，序列中较前的输入会逐渐被遗忘，这对模型的效果会造成很大的影响。

基本原理:

LSTM:

- 最主要的一层是$g_t$,这层和基础RNN cell基本相同，都是用过$x_t$和$h_{t-1}$得出结果。
在此基础上还有三种gate contrillers，用sigmoid函数输出0-1的结果来控制输入输出。
- forget gate 控制长期记忆中的哪些部分需要舍弃
- input gate 控制 $g_t$中的哪些部分需要加到长期记忆中
- output gate 控制更新后的长期记忆哪一部分输出到$h_t$,$y_t$中

GRU:

- GRU是LSTM的精简版，它把长短期记忆都融合成一个状态$h_t$
- 由一个controller同时控制forget gate和input gate
- 没有output gate 状态会根据序列时间改变


#### Q21: Attentional RNN 以及 Stacked RNN 和 Bi-RNN 分别是什么，其做了什么改动？
回答：

Attentional RNN:  

在计算输出值时的在考虑原有输入的同时，加入了context vector $s_{i-1} = f(s_{i-1},y_{i-1},c_i)$,其中$c_i$是源输入的$h_t$的加权平均$c_i = \sum_{j=1}^{T_x}\alpha_{ij}h_j$,而$\alpha_{ij} = softmax(e_{ij})$, $e_{ij} = a(s_{i-1},h_j)$,其中a是一个全链接神经网络。这样做可以建立在计算输出时，建立与源输入的联系，并且专注与这一时刻有用的信息。  

Stacked RNN:

可以理解为RNN cell的堆叠，t时刻的的输出不仅仅是t+1时刻的输入，同时候也是下一层RNN cell的输入，这样可以根据需要增加网络深度。  

Bi-RNN:

在原有RNN的基础上，计算从后往前的RNN cell，t时刻的预测值就包含了输入序列的前后信息。t时刻的预测可表示为$\hat{y_t} = \sigma_y(w_y[\overrightarrow{h_t},\overleftarrow{h_t}]+b_y)$

## 第二部分： 实验过程

### IIX 数据预处理

#### Q22：要实现文本分类或情感分类，文本信息需要进行哪些初始化操作？自己手工实现，keras提供的API，tenorflow提供的API，分别是哪些？请提供关键代码置于下边`回答`中

回答：

1. 简述所需要的初始化操作：  
    + 切分词，可根据需要去掉停用词或标点符号
    + 构造word-id，id-word的hash表
    + 用构造的hash表把句子转化成整数序列
    + 准备好预训练的embedding(optional)
    + 构造batch函数决定如何输入训练数据  
    
切分词使用jieba分词

keras提供的api：
+ keras.preprocessing.text 提供Tokenizer类可以文本构造需要的hash表并把句子转化为整数序列
+ keras.preprocessing.sequence 提供pad_sequence函数为序列提供padding
+ mini-batch：keras在训练时只需要指定batch size即可，也可以根据自己的需要写一个generator输入训练数据

tensorflow提供的api： 
+ tensorflow 集成了keras的api，对于文本处理大同小异
+ 训练时需要自定义batch函数，一般是写一个generator生成需要的训练数据


2. 自己手工实现的id_to_word, word_to_id, padding, batched等操作如何实现？
    + id_to_word, word_to_id
    
    ```python
        word_count = Counter()
    for s in movie_comments['word_sequence']:
        for w in s.split():
            word_count[w] += 1

    id_to_word = {}
    id_to_word[0] = '<PAD>'
    for i,w in enumerate(word_count,1):id_to_word[i] = w

    word_to_id = {w:i for i,w in id_to_word.items()}```
    + padding
    
    ```python
    def pad_sequence(word_sequence,maxlength):
    
        pad_sequence = np.zeros((len(word_sequence),maxlength),dtype=np.int32)
        for ri,row in enumerate(word_sequence):
            for ci,w in enumerate(row.split()[:maxlength]):
                pad_sequence[ri,ci] = word_to_id[w]
        return pad_sequence
        
    sequence = pad_sequence(movie_comments['word_sequence'],200)
    ```
    
    + batched
    
    ```python
    star = movie_comments['star'].values
    star = star.astype(np.int32)
    sub_one = np.vectorize(lambda x:x-1)
    star = sub_one(star)

    onehot_star = np.eye(5)[star]

    def get_sequence_batches(datasets,labels,batch_size):
        assert datasets.shape[0] == labels.shape[0]
        batch_num = datasets.shape[0] // batch_size
        datasets_cut = datasets[:batch_num*batch_size,:]
        last_datasets = datasets[batch_num*batch_size:,:]
        labels_cut = labels[:batch_num*batch_size,:]
        last_labels = labels[batch_num*batch_size:,:]
    
    for i in range(batch_num+1):
        if i == batch_num:
            yield last_datasets,last_labels
            break
        x = datasets_cut[i*batch_size:(i+1)*batch_size,:]
        y = labels_cut[i*batch_size:(i+1)*batch_size,:]
        yield x,y
    ```

hints: 

+ 参考1 https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py
+ 参考2 

---

## code for sentimental analysis(preprocessing)


In [311]:
import pandas as pd
import jieba
from opencc import OpenCC
import re
from collections import Counter

In [312]:
def remove_char(string):
    return re.sub('[a-zA-Z0-9]','',string)

In [313]:
def t2s(string):
    opencc = OpenCC('t2s')
    return opencc.convert(string)

### word to id,id to word

In [314]:
movie_comments = pd.read_csv('./movie_comments.csv')

/Users/mozhiwen/anaconda3/envs/nlp/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [315]:
movie_comments = movie_comments.dropna(subset=['comment','star'])

In [316]:
movie_comments = movie_comments.drop(568,axis=0)

In [317]:
movie_comments.head()

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1


In [318]:
def cut(string): return ' '.join([w for w in jieba.cut(string)])

In [319]:
movie_comments['word_sequence'] = movie_comments.comment.apply(remove_char)

In [320]:
movie_comments['word_sequence'] = movie_comments.word_sequence.apply(t2s)

In [321]:
movie_comments['word_sequence'] = movie_comments.word_sequence.apply(cut)

In [322]:
movie_comments.to_csv('./movie_comments_data.csv')

In [323]:
word_count = Counter()
for s in movie_comments['word_sequence']:
    for w in s.split():
        word_count[w] += 1

In [324]:
word_count

Counter({'吴京': 279,
         '意淫': 281,
         '到': 10373,
         '了': 102381,
         '脑残': 319,
         '的': 328466,
         '地步': 197,
         '，': 353805,
         '看': 34250,
         '恶心': 943,
         '想': 7474,
         '吐': 574,
         '首映礼': 42,
         '。': 219514,
         '太': 13043,
         '恐怖': 599,
         '这个': 10282,
         '电影': 34614,
         '不讲道理': 8,
         '完全': 4152,
         '就是': 14015,
         '在': 31161,
         '实现': 270,
         '他': 10663,
         '小': 6649,
         '粉红': 39,
         '英雄': 1706,
         '梦': 885,
         '各种': 3143,
         '装备': 83,
         '轮番': 21,
         '上场': 17,
         '视': 29,
         '物理': 63,
         '逻辑': 1417,
         '于': 1783,
         '不顾': 57,
         '不得不': 670,
         '说': 11120,
         '有钱': 205,
         '真': 5188,
         '好': 23131,
         '随意': 170,
         '胡闹': 45,
         '炒作': 70,
         '水平': 819,
         '不输': 48,
         '冯小刚': 266,
         '但小刚': 1,
       

In [325]:
id_to_word = {}
id_to_word[0] = '<PAD>'
for i,w in enumerate(word_count,1):id_to_word[i] = w

In [326]:
word_to_id = {w:i for i,w in id_to_word.items()}

### padding

In [327]:
def pad_sequence(word_sequence,maxlength):
    pad_sequence = np.zeros((len(word_sequence),maxlength),dtype=np.int32)
    for ri,row in enumerate(word_sequence):
        for ci,w in enumerate(row.split()[:maxlength]):
            pad_sequence[ri,ci] = word_to_id[w]
    return pad_sequence

In [341]:
sequence = pad_sequence(movie_comments['word_sequence'],200)

### batches

In [329]:
star = movie_comments['star'].values
star = star.astype(np.int32)
sub_one = np.vectorize(lambda x:x-1)
star = sub_one(star)

In [330]:
u,c = np.unique(star,return_counts=True)

In [331]:
for s,n in zip(u,c):print(s,n/sum(c))

0 0.09393332160584947
1 0.10759711503896839
2 0.2511415175874016
3 0.3204968374035351
4 0.22683120836424545


In [332]:
onehot_star = np.eye(5,dtype=np.int32)[star]

In [333]:
def get_sequence_batches(datasets,labels,batch_size):
    assert datasets.shape[0] == labels.shape[0]
    batch_num = datasets.shape[0] // batch_size
    datasets_cut = datasets[:batch_num*batch_size,:]
    last_datasets = datasets[batch_num*batch_size:,:]
    labels_cut = labels[:batch_num*batch_size,:]
    last_labels = labels[batch_num*batch_size:,:]
    
    for i in range(batch_num+1):
        if i == batch_num:
            yield last_datasets,last_labels
            break
        x = datasets_cut[i*batch_size:(i+1)*batch_size,:]
        y = labels_cut[i*batch_size:(i+1)*batch_size,:]
        yield x,y

### IX 构建神经网络模型

#### Q22:在没有预训练的词向量时候， keras 如何实现embedding操作，即如何依据一个单词的序列获得其向量表示？
回答：
```python
max_feature = 200
embed_size =300
vocab_size = len(vocab_to_id)

from keras.models import Sequential
from keras.layers import Embedding

model = Sequential()
model.add(Embedding(vocab_size,embed_size))

```

#### Q23:在没有预训练的词向量时候， tensorflow 如何实现embedding操作，即如何依据一个单词的序列获得其向量表示？
回答：
```python
import tensorflow as tf

input_seq = tf.Placeholder(tf.int32,[None,max_feature])

embedding = tf.Variable(tf.random_uniform((vocab_size,embed_size),-1,1))

embed = tf.nn.embedding_lookup(embedding,input_seq)
```

#### Q24: 在有预先训练的词向量时候，keras和tensorflow又如何实现embeding操作

回答：
keras:
```python
pretrained_vec = ... # is a pretrained embedding numpy ndarray

import keras
from keras.models import Model,Sequential
from keras.layers import Input,Dense,Embedding

embeddings_initializer = keras.initializers.Constant(value=pretrained_vec)

inputs = Input(shape=(max_feature,))
embed_layer = Embedding(vocab_size,embed_size,embeddings_initializer=embeddings_initializer)(inputs)
embed_layer.trainable = False
```

tensorflow:
```python
pretrained_vec = ... # is a pretrained embedding numpy ndarray

import tensorflow as tf

input_seq = tf.Placeholder(tf.int32,[None,max_feature])

embedding = tf.get_variable(name="embedding", shape=pretrained_vec.shape, initializer=tf.constant_initializer(pretrained_vec), trainable=False)

embed = tf.nn.embedding_lookup(embedding,input_seq)
```

#### Q25： 基于上文进行的数据预处理，使用keras和tensorflow如何构建神经网络模型？请提供关键代码

回答： keras模型构建的关键代码：

回答：tensorflow模型构建的关键代码：

###  X 使用keras的history观察loss以及accuracy的变化

#### Q26: keras如何观察模型的loss变化以及准确率的变化，请列出关键代码

回答：

#### Q27: 请使用matplotlib画出loss变化的趋势；

回答：

### XI 使用tensorflow tensorboard观察loss, accuracy的变化

#### Q28: tensorflow如何观察模型的loss变化以及准确率的变化， tensor board 如何使用？ 请列出关键代码

回答：

#### Q29: 试着点击tensor board的不同按钮 观察图像的变化； 试着给tensorflow board机制 写入不同时候训练的模型时候，给模型取不同的名字，观察tensor board的图像变化；

回答：

### XII 观察熟悉RNN的两种变体的原理和方法

#### Q30:试着改进RNN，使用LSTM， GRU 进行模型的改动， 观察训练结果(loss和accuracy)的变化， 你观察到了什么变化？ 如何解释？

回答：

---

## hyper params

In [350]:
max_feature = 200
embed_size =300
vocab_size = len(word_to_id)
lstm_size = 256
dropout_rate = 0.2
batch_size =32
learning_rate = 0.001
epochs =20

## split the data

In [343]:
split_frac = 0.8

from sklearn.utils import shuffle

sequence,onehot_star = shuffle(sequence,onehot_star,random_state=42)

split_idx = int(len(sequence)*0.8)

train_x, val_x = sequence[:split_idx], sequence[split_idx:]
train_y, val_y = onehot_star[:split_idx], onehot_star[split_idx:]

test_idx = int(len(val_x)*0.5)

val_x, test_x = val_x[:test_idx], val_x[test_idx:]
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

print('train_x shape:', train_x.shape)
print('val_x shape:', val_x.shape)
print('test_x shape:', test_x.shape)

train_x shape: (209195, 200)
val_x shape: (26149, 200)
test_x shape: (26150, 200)


## load embeddings

In [240]:
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('./cc.zh.300.vec')

/Users/mozhiwen/anaconda3/envs/nlp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [336]:
count_not_in_pretrain = 0
word_not_in_pretrain = []
np.random.seed(42)
pretrianed_wv = np.random.uniform(-1,1,(vocab_size,embed_size))
for word,idx in word_to_id.items():
    if word not in word_vectors:
        count_not_in_pretrain += 1
        word_not_in_pretrain.append(word)
        continue
    pretrianed_wv[idx] = word_vectors[word]

In [344]:
pretrianed_wv.shape

(125795, 300)

In [355]:
pretrianed_wv.dump('./pretrained_wv.npy')

In [340]:
word_not_in_pretrain

['<PAD>',
 '不讲道理',
 '但小刚',
 '中二得',
 '虽远必',
 '一百倍',
 '第一集',
 '第二集',
 '想做到',
 '弹簧床',
 '开太大',
 '弥漫着',
 '那颗',
 '跳个',
 '像杰',
 '吴京路',
 '几处',
 '挺燃',
 '太违',
 '第一部',
 '好太多',
 '没见识',
 '戏可算',
 '狂用',
 '不功',
 '有略',
 '吓了一跳',
 '脑子里',
 '戏看',
 '太假',
 '越做越',
 '黑鹰坠落',
 '但论',
 '角色定位',
 '那套',
 '张翰太违',
 '目瞪狗',
 '人牛',
 '挺差',
 '没太多',
 '吴京脑',
 '戏撑',
 '两星',
 '完成度',
 '吴京会',
 '特种部队',
 '补大',
 '无亮点',
 '张翰变',
 '子当',
 '画内',
 '对龙小云',
 '血厚到',
 '红血',
 '投机取巧',
 '假瞎燃',
 '吴京直',
 '🇨',
 '🇳',
 '一下张',
 '可演',
 '坠露',
 '第三部',
 '好燃',
 '庇衣',
 '意料之中',
 '意料之外',
 '网红拉',
 '弹弹琴',
 '如入无人之境',
 '一万倍',
 '满到',
 '张翰脸',
 '女主白',
 '七分',
 '不招',
 '牛逼到',
 '装逼装',
 '片有',
 '已三刷',
 '英雄人物',
 '发动战争',
 '这苦',
 '之力',
 '跪久',
 '站不起来',
 '双重标准',
 '玩爱',
 '国产电影',
 '燃不燃',
 '这一星',
 '四张',
 '卢靖姗真',
 '健康美',
 '连特',
 '长驱直入',
 '多牛',
 '带你去',
 '战争场面',
 '减一星',
 '剪一星',
 '全程无',
 '比战',
 '点分',
 '那句话',
 '压人压',
 '成肉',
 '吴京站',
 '继续前进',
 '第二部',
 '很燃',
 '看得人',
 '本有',
 '有点像',
 '总立',
 '半红',
 '靠战',
 '毫无惧色',
 '双标',
 '亿直',
 '打差',
 '评是',
 '几把',
 '谁他妈',
 '自我认识',
 '精神分裂',
 '一屏',
 '一星',
 '虽多

In [338]:
print('word not in pretrained vocab: %.2f' % (count_not_in_pretrain/vocab_size))

word not in pretrained vocab: 0.38


## code for sentimental analysis(construct model keras)

In [356]:
import keras
from keras.layers import Input,Embedding,Dense,LSTM
from keras.models import Model,Sequential
from keras.optimizers import adam
from keras.initializers import Constant

In [357]:
inputs = Input(shape=(max_feature,))

weights = Constant(value=pretrianed_wv)
embed_layer = Embedding(vocab_size,embed_size,embeddings_initializer=weights)
embed_layer.trainable = False

embed = embed_layer(inputs)

LSTM_layer = LSTM(lstm_size,dropout=dropout_rate,recurrent_dropout=dropout_rate)(embed)

dense_layer1 = Dense(128, activation='sigmoid')(LSTM_layer)
dense_layer2 = Dense(64 ,activation='sigmoid')(dense_layer1)
out_ = Dense(5, activation='softmax')(dense_layer2)

In [358]:
sentiment_model = Model(inputs=inputs,outputs=out_)
sentiment_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 200, 300)          37738500  
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               570368    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_8 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_9 (Dense)              (None, 5)                 325       
Total params: 38,350,345
Trainable params: 611,845
Non-trainable params: 37,738,500
__________________________________________________________

In [359]:
opt = adam(lr=learning_rate)

In [360]:
sentiment_model.compile(loss='categorical_crossentropy',
                        optimizer=opt,
                        metrics=['accuracy'])

In [361]:
history = sentiment_model.fit(train_x,train_y,batch_size=batch_size,
                             epochs=epochs,
                             validation_data=(val_x,val_y),
                             verbose=1)

Train on 209195 samples, validate on 26149 samples
Epoch 1/20
 10112/209195 [>.............................] - ETA: 1:23:15 - loss: 1.5178 - acc: 0.3099

KeyboardInterrupt: 

In [ ]:
score,acc = sentiment_model.evaluate(test_x,test_y,
                                    batch_size=batchsize)
print('Test score:', score)
print('Test accuracy', acc)

## code for sentimental analysis(construct model tensorflow)

### XIII 模型的改进

#### Q31: 修改vocabulary size, embedding size, 并且结合使用LSTM， GRU， Bi-RNN， Stacked， Attentional, regularization, 等各种方法组合进行模型的优化， 至少进行10次优化，每次优化请按照以下步骤填写：

回答：

---这是一个实例----

第1次优化：

1. 存在的问题： loss下降太慢；
2. 准备进行的优化：减小模型的神经单元数量；
3. 期待的结果：loss下降加快；
4. 实际结果：loss下降的确加快(或者并没有加快)
5. 原因分析：模型神经元数量减小，收敛需要的次数减少，loss下降加快


---你的实验优化结构记录在此---

**第1次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第2次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第3次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第4次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第5次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第6次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第7次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第9次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第10次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：


### XIV问题： 本次实验的总结

请写实验的总结报告，描述此次项目的主要过程，其中遇到的问题，以及如何解决这些问题的，以及有什么经验和收获。